<a href="https://colab.research.google.com/github/zakihir0/jigsaw---Agile-Community-Rules-Classification/blob/main/Jigsaw_%7C_Qwen3_4B_Training_one_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

jigsaw_agile_community_rules_path = kagglehub.competition_download('jigsaw-agile-community-rules')

print('Data source import complete.')


100%|██████████| 695k/695k [00:00<00:00, 760kB/s]

Extracting files...
Data source import complete.


In [3]:
!pip install -U unsloth
!pip install -U torchvision

import os
from google.colab import userdata

WANDB_API = userdata.get('WANDB_API_KEY')
WANDB_PROJECT = userdata.get('WANDB_PROJECT')
os.environ['WANDB_API_KEY'] = WANDB_API
os.environ['WANDB_PROJECT'] = WANDB_PROJECT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 134.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# === 改善版：追加ヘッド + AUC最適化設定 =========================
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template
import torch, torch.nn as nn
import os, warnings
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, TrainerCallback
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, precision_recall_fscore_support

warnings.filterwarnings("ignore")

# ---------------------- config ----------------------
max_seq_length = 2048
dtype = None
load_in_4bit = True
SEED = 3407
EVAL_STEPS = 50
VALID_RATIO = 0.1
LABEL_COL = "rule_violation"
np.random.seed(SEED); torch.manual_seed(SEED)

# ---------------------- safetensors共有解除 ----------------------
def untie_embeddings_for_safetensors(model):
    m = model
    for a in ["base","base_model","model"]:
        if hasattr(m,a): m = getattr(m,a)
    embed = getattr(getattr(m,"model",m), "embed_tokens", None)
    lm_head = getattr(m, "lm_head", None)
    if embed is None or lm_head is None: return model
    if lm_head.weight.data_ptr() == embed.weight.data_ptr():
        lm_head.weight = nn.Parameter(lm_head.weight.detach().clone())
        print("[untie] lm_head weight cloned (embed tie removed).")
    if hasattr(m, "config"): m.config.tie_word_embeddings = False
    if hasattr(m, "tie_weights"): m.tie_weights = lambda *a, **kw: None
    return model

class UntieBeforeSaveCallback(TrainerCallback):
    def on_save(self, args, state, control, **kw):
        tr = kw.get("trainer")
        if tr: tr.model = untie_embeddings_for_safetensors(tr.model); return control
    def on_train_begin(self, args, state, control, **kw):
        tr = kw.get("trainer")
        if tr: tr.model = untie_embeddings_for_safetensors(tr.model); return control

# ---------------------- model & tokenizer ----------------------
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen3-4B", max_seq_length=max_seq_length, dtype=dtype, load_in_4bit=load_in_4bit
)
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth", random_state=SEED,
    use_rslora=False, loftq_config=None,
)

# ---------------------- data ----------------------
system_prompt = (
    "You are given a comment on reddit and a rule. "
    "Your task is to classify whether the comment violates the rule. Only respond Yes/No."
)
train_df = pd.read_csv(os.path.join(jigsaw_agile_community_rules_path, "train.csv"))
if train_df[LABEL_COL].dtype != int:
    mp = {"No":0,"no":0,"Yes":1,"yes":1, False:0, True:1}
    train_df[LABEL_COL] = train_df[LABEL_COL].map(mp).fillna(train_df[LABEL_COL]).astype(int)

user_prompt = (
    "Subreddit: r/{subreddit}\n"
    "Rule: {rule}\n"
    "Examples:\n"
    "1) {positive_example_1}\nViolation: Yes\n\n"
    "2) {negative_example_1}\nViolation: No\n\n"
    "3) {negative_example_2}\nViolation: No\n\n"
    "4) {positive_example_2}\nViolation: Yes\n"
    "Comment:\n{body}\n"
    "Violation: "
)

def build_convs(df):
    convs=[]
    for r in df.itertuples(index=False):
        convs.append([
            {"role":"system","content":system_prompt},
            {"role":"user","content":user_prompt.format(
                subreddit=getattr(r,"subreddit",""),
                rule=getattr(r,"rule",""),
                positive_example_1=getattr(r,"positive_example_1",""),
                positive_example_2=getattr(r,"positive_example_2",""),
                negative_example_1=getattr(r,"negative_example_1",""),
                negative_example_2=getattr(r,"negative_example_2",""),
                body=getattr(r,"body",""),
            )}
        ])
    return convs

tr_df, va_df = train_test_split(
    train_df, test_size=VALID_RATIO, random_state=SEED, stratify=train_df[LABEL_COL].astype(int)
)
train_texts = [tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False) for c in build_convs(tr_df)]
valid_texts = [tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False) for c in build_convs(va_df)]
dds = DatasetDict({
    "train": Dataset.from_dict({"text":train_texts,"label":tr_df[LABEL_COL].astype(int).values}),
    "valid": Dataset.from_dict({"text":valid_texts,"label":va_df[LABEL_COL].astype(int).values}),
})
def tok_fn(b):
    enc = tokenizer(b["text"], padding=True, truncation=True, max_length=max_seq_length)
    enc["labels"]=b["label"]; return enc
dds = dds.map(tok_fn, batched=True, remove_columns=["text","label"])
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ---------------------- wrap: 既存lm_headを残しつつ新ヘッドを追加 ----------------------
class CausalWithExtraHead(nn.Module):
    def __init__(self, base_model, hidden_size, num_labels=2, dropout=0.1, head_name="moderation_head"):
        super().__init__()
        self.base = base_model
        self.dropout = nn.Dropout(dropout)
        self.moderation_head = nn.Linear(hidden_size, num_labels)
        if hasattr(self.base, "model") and hasattr(self.base.model, "config"):
            self.base.model.config.cls_head_name = head_name
        self._head_name = head_name

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        out = self.base.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            use_cache=False,
        )
        last = out.hidden_states[-1]
        last_idx = attention_mask.sum(dim=1) - 1
        h_last = last[torch.arange(last.size(0)), last_idx]
        logits = self.moderation_head(self.dropout(h_last))
        loss = nn.CrossEntropyLoss()(logits, labels.long()) if labels is not None else None
        return {"loss": loss, "logits": logits}

hidden = model.model.config.hidden_size
clf_model = CausalWithExtraHead(model, hidden_size=hidden, num_labels=2, dropout=0.1)

# ---------------------- metrics ----------------------
def compute_metrics(eval_pred):
    try:
        logits, labels = eval_pred
    except Exception:
        logits, labels = eval_pred.predictions, eval_pred.label_ids
    probs = torch.tensor(logits).softmax(-1)[:,1].cpu().numpy()
    preds = (probs>=0.5).astype(int)
    p,r,f1,_ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    return {
        "auc": float(roc_auc_score(labels, probs)),
        "f1": float(f1),
        "acc": float(accuracy_score(labels, preds)),
        "logloss": float(log_loss(labels, np.clip(probs,1e-6,1-1e-6))),
        "precision": float(p), "recall": float(r)
    }

# ---------------------- train ----------------------
args = TrainingArguments(
    output_dir="outputs_cls_extrahead",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,          # ← 勾配情報量UP
    learning_rate=2e-4,                     # ← 初期LRやや上げる
    num_train_epochs=4,                     # ← エポック倍増
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,                       # ← ウォームアップ延長
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    logging_steps=EVAL_STEPS,
    save_strategy="steps",
    save_safetensors=True,
    optim="paged_adamw_8bit",
    weight_decay=0.005,                     # ← 正則化緩和
    max_grad_norm=1.0,
    seed=SEED,
    report_to="wandb",
    load_best_model_at_end=True,            # ← AUC最良モデルを自動選択
    metric_for_best_model="auc",
    greater_is_better=True,
)

trainer = Trainer(
    model=clf_model,
    args=args,
    train_dataset=dds["train"],
    eval_dataset=dds["valid"],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.model = untie_embeddings_for_safetensors(trainer.model)
trainer.add_callback(UntieBeforeSaveCallback())

trainer.train()
print(trainer.evaluate())

trainer.model = untie_embeddings_for_safetensors(trainer.model)
trainer.save_model("outputs_cls_extrahead/best")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.9.0+cu128)
    Python  3.9.23 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.9.0+cu128)
    Python  3.9.23 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.4: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

[untie] lm_head weight cloned (embed tie removed).


wandb: Currently logged in as: zakihiro to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
Unsloth: Not an error, but CausalWithExtraHead does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,Auc,F1,Acc,Logloss,Precision,Recall
50,0.903600,1.120977,0.714175,0.673203,0.507389,1.120764,0.507389,1.000000
100,0.718500,0.590597,0.809951,0.770563,0.738916,0.590678,0.695312,0.864078


In [ ]:
from google.colab import runtime
runtime.unassign()